![](./images/求state-value的两种办法.png)
![](./images/矩阵表示求解vpi.png)
![](./images/矩阵表示求解vpi1.png)
### [github课本地址](https://github.com/MathFoundationRL/Book-Mathmatical-Foundation-of-Reinforcement-Learning.git)

In [1]:
import numpy as np
n = 4
I = np.identity(n)
print(f"I:{I}")

gamma = np.random.uniform(0,1)
print(f'gamma{gamma}')

P = np.ones((n,n))
# for i in range(n):
#     single_step = abs(np.random.randn(1,n))
#     second = single_step/single_step.sum()
#     P[i]=second
# P = np.random.random_integers(0,1,size=n*n).reshape((n,n))
print(f"P:{P}")
r = np.array([1,0,0,0])#np.random.normal(0,10,n)
print(f"r:{r}")

# pi = np.zeros((n,n))
# for i in range(n): 
#     single_tep = abs(np.random.normal(0.1,1,n))
#     second = single_tep/single_tep.sum()
#     pi[i]= second 

single_step = np.identity(n)
np.random.shuffle(single_step)
pi = single_step    
print(f"pi:{pi}")

I:[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
gamma0.14471789883353492
P:[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
r:[1 0 0 0]
pi:[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [2]:
pi.dot(r)

array([0., 0., 0., 1.])

矩阵解析法

In [3]:
P_pi = P*pi
r_pi = r.dot(pi)
print(f"r_pi:{r_pi}")
v_pi = np.linalg.inv(I-gamma*P_pi).dot(r_pi)
%time v_pi

r_pi:[0. 0. 0. 1.]
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


array([0.1478136 , 0.        , 0.        , 1.02139127])

验证v是否正确

In [4]:
v_pi = (r.dot(pi))+(gamma*(P*pi).dot(v_pi))
v_pi

array([0.1478136 , 0.        , 0.        , 1.02139127])

迭代法求state value

In [5]:
dim_of_v = v_pi.size
vpi0 = np.full((dim_of_v),0)
max_iter = 10000
histroy_v_pi = np.zeros((max_iter+1,dim_of_v))
for i in range(0,max_iter):
    if i ==0:
        v_pi_k = vpi0
    P_pi = P*pi
    r_pi = r.dot(pi)
    v_pi_k1 = r_pi + gamma * P_pi.dot(v_pi_k)
    # print(v_pi_k)
    v_pi_k = v_pi_k1
    histroy_v_pi[i+1] = v_pi_k
    if (histroy_v_pi[i+1][-1]-histroy_v_pi[i])[-1]==0 :
        break
%time v_pi_k,i

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.58 µs


(array([0.1447179, 0.       , 0.       , 1.       ]), 1)

#### 整理成 contractive function 求state value

In [6]:
def Vpi(v_pi_k,pi,r,gamma,P):
    P_pi = P*pi
    r_pi = r.dot(pi)
    v_pi_k1 = r_pi  + gamma * P_pi.dot(v_pi_k)
    return v_pi_k1

def cotraction_mapping_for_Vpi1(pi,r,gamma,P,max_iter=10000):
    dim_of_v = r.size
    vpi0 = np.full(dim_of_v,0)
    #histroy_v_pi = np.zeros((max_iter+1,dim_of_v))
    for i in range(0,max_iter):
        if i ==0:
            v_pi_k = vpi0
        v_pi_k1 = Vpi(v_pi_k,pi,r,gamma,P)
        v_pi_k = v_pi_k1
        histroy_v_pi[i+1] = v_pi_k
        if (Vpi(v_pi_k,pi,r,gamma,P)-v_pi_k).sum()==0:
            break
        # if (histroy_v_pi[i+1]-histroy_v_pi[i]).sum()==0 :
        #     print(Vpi_k(v_pi_k,pi,r,gamma,P))
            break
#  
    return v_pi_k,i

def cotraction_mapping_for_Vpi2(pi,r,gamma,P,max_iter=10000):
    dim_of_v = r.size
    vpi0 = np.full(dim_of_v,0)
    histroy_v_pi = np.zeros((max_iter+1,dim_of_v))
    for i in range(0,max_iter):
        if i ==0:
            v_pi_k = vpi0
        v_pi_k1 = Vpi(v_pi_k,pi,r,gamma,P)
        v_pi_k = v_pi_k1
        histroy_v_pi[i+1] = v_pi_k
        # if (Vpi(v_pi_k,pi,r,gamma,P)-v_pi_k).sum()==0:
        #     break
        if (histroy_v_pi[i+1]-histroy_v_pi[i]).sum()==0 :
            # print(Vpi(v_pi_k,pi,r,gamma,P))
            break
#  
    return v_pi_k,i

%time cotraction_mapping_for_Vpi1(pi,r,gamma,P)

CPU times: user 539 µs, sys: 110 µs, total: 649 µs
Wall time: 458 µs


(array([0.1478136 , 0.        , 0.        , 1.02139127]), 19)

In [7]:
%time cotraction_mapping_for_Vpi2(pi,r,gamma,P)

CPU times: user 496 µs, sys: 103 µs, total: 599 µs
Wall time: 405 µs


(array([0.1478136 , 0.        , 0.        , 1.02139127]), 20)

### 求解 $q_{k}(s,a)$
### $$ q_{k}(s,a) = \sum_{r} p(r|s,a) r + \gamma \sum_{s^{'}} p(s^{'} | s,a ) v_{k}(s^{'}) $$
![](./images/q_table的例子1.png)
![](./images/q_table的例子2.png)
### 设 $ k=1，v0 = 1，\gamma = 0.9 $
### $ q(s1,a_{l} ) = -1+0.9*1 = -0.1$
![](./images/q_table的例子3.png)


In [8]:
import numpy as np


#### 求贝尔曼最优的$\pi$

In [9]:


# vk= (r.dot(pi))+(gamma*P.dot(pi).dot(v_pi))
# print(f"v_pi:{v_pi}")

# def pi_func(pi,r,gamma,P,v_pi_k):
#     P_pi = P*pi
#     r_pi = r.dot(pi)
#     v_pi_k1 = r_pi  + gamma * P_pi.dot(v_pi_k)
#     return v_pi_k,i


# def solve_pi(r,gamma,P,vk):
#     pi0 = np.ones_like(P)
#     dim = r.size
#     pi0 = np.ones_like(P)/n
#     histroy_pi = np.zeros((max_iter+1,dim))
#     for i in range(0,max_iter):
#         if i ==0:
#             pi_k = pi0
#         pi_k1 = pi_func(pi,r,gamma,P,vk)
#         pi_k = pi_k1

#     # qk ,i = cotraction_mapping_for_Vpi2(pi,r,gamma,P)
#     #solve_pi = np.where(qk==qk.max(),True,False).astype(int)
#     return pi_k1

# pi = solve_pi(pi,gamma,P,vk)
# pi


In [10]:
v_pi -v_pi_k

array([0.0030957 , 0.        , 0.        , 0.02139127])

In [11]:
histroy_v_pi[i]

array([0., 0., 0., 1.])

#### 这个函数之所以起作用，就是因为他是一个contraction mapping,以下再用一个简单的函数证实它

In [12]:
def f(x):
    return x*np.sin(0.5)

xn = 10000
x0 = 1
histroy_xk = np.zeros(xn+1)
for i in range(xn):
    if i == 0:
        x = x0
    x_k1 = f(x)
    x = x_k1
    
    histroy_xk[i]=x
    if f(x) == x:
        break
    # if abs(histroy_xk[-1]-histroy_xk[-2])==0:
    #     break    
x_k1,i
    

(0.0, 1012)

def f(x)